# Retrieving queries from the vector database

In [2]:
import pandas as pd
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.core.vector_stores import VectorStoreQuery
from dotenv import load_dotenv, find_dotenv
import os
import psycopg2
from llama_index.vector_stores.postgres import PGVectorStore
from llama_index.core.schema import NodeWithScore
from typing import Optional
from vector_db_retriever import VectorDBRetriever
from llama_index.core import Settings

## Loading the LLM and sentence transformers

In [2]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

In [12]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_path = "C:/Users/Ugne/AppData/Local/llama_index/models/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    model_path=model_path,
    temperature=0.7,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from C:/Users/Ugne/AppData/Local/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 

## Initialising the vector store

In [4]:
load_dotenv(find_dotenv())
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [5]:
db_name = "vectors_db"
host = "localhost"
password = os.getenv("POSTGRES_PW")
port = "5432"
user = "postgres"

conn = psycopg2.connect(
    dbname="postgres",
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="llama2_paper",
    embed_dim=384,  # openai embedding dimension
)

# Retrieval

##  Parse Result into a Set of Nodes

In [6]:
def get_top_result_from_vector_store(query_str, vector_store, query_mode="default"):
    query_embedding = embed_model.get_query_embedding(query_str)
    vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
    )
    query_result = vector_store.query(vector_store_query)
    print(query_result.nodes[0].get_content())
    return query_result

In [7]:
query_str = "Tell me about the Vilnius TV tower"
query_result = get_top_result_from_vector_store(query_str, vector_store, query_mode="default")

After the renewed premises of the Vilnius TV Tower were opened on Tuesday, it is now possible to rent apartments at the historic building for between 796 and 954 euros per night.The Vilnius TV Tower now has a refurbished bar, a meeting room, and two apartments, called Upė (River) and Miestas (City). According to the booking.com website, the apartments can be booked from next week. They are rented directly to customers by the state-owned Lithuanian Radio and Television Centre (Telecentras).According to Valdas Kaminskas, head of communications at Telecentras, the apartments oversee different panoramas of the city.“One view is directed towards Vingis Park, while the other is more towards the city. The apartments are dominated by an oval shape, which echoes the roundness of the TV Tower,” he said.Asked about the price, Kaminskas noted that apartments are aimed at an exclusive segment of consumers.“These are the people who can either afford or are curious and can rent, try, experience a cer

In [8]:
nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))

In [9]:
nodes_with_scores

[NodeWithScore(node=TextNode(id_='d29acc21-137e-4e24-a4c4-ebd8f932b943', embedding=None, metadata={'source_url': ''}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='After the renewed premises of the Vilnius TV Tower were opened on Tuesday, it is now possible to rent apartments at the historic building for between 796 and 954 euros per night.The Vilnius TV Tower now has a refurbished bar, a meeting room, and two apartments, called Upė (River) and Miestas (City). According to the booking.com website, the apartments can be booked from next week. They are rented directly to customers by the state-owned Lithuanian Radio and Television Centre (Telecentras).According to Valdas Kaminskas, head of communications at Telecentras, the apartments oversee different panoramas of the city.“One view is directed towards Vingis Park, while the other is more towards the city. The apartments are dominated by an oval shape, which echoes the roundness of the TV Tower,

## Plugging into RetrieverQueryEngine

In [ ]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

query_str = "Tell me about the Vilnius TV tower"

response = query_engine.query(query_str)

Batches: 100%|██████████| 1/1 [00:06<00:00,  6.54s/it]


In [4]:
print(str(response))

NameError: name 'response' is not defined